# Metaprogramming

"Meta": one level up

"Metaprogramming": programming a program, i.e. write code that writes code.

Means talking about Julia code within Julia! (Because Julia is a homoiconic language, i.e. code is data (type) and data is code.)

Refs: see lecture [slides](https://github.com/alanedelman/18.337_2016/blob/master/lectures/lecture_5_09_21/Introduction%20to%20metaprogramming.ipynb) by Alan Edelman for more.

Also, watch and follow [this video](https://www.youtube.com/watch?v=e6LGMeoQhfs) for a concise introduction on `macros`.

## 1. A motivation case of metaprogramming

* To avoid repetitive code

In [23]:
struct MyInteger
    a::Int
end

In [ ]:
# No meta-programming, just define the methods
+(x::MyInteger, y::MyInteger) = MyInteger(x.a + y.a)
-(x::MyInteger, y::MyInteger) = MyInteger(x.a - y.a)
*(x::MyInteger, y::MyInteger) = MyInteger(x.a * y.a)
# ...

In [32]:
# Use meta-programming
import Base: +,-, *

for op in [:+, :-, :*]
    @eval $op(x::MyInteger, y::MyInteger) = MyInteger($op(x.a, y.a))
    # Equivalent to the following:
    # eval(:($op(x::MyInteger, y::MyInteger) = MyInteger($op(x.a, y.a))))
    # A common mistake is to use MyInteger(x.a $op y.a) as follows:
    # @eval ($op)(x::MyInteger, y::MyInteger) = MyInteger(x.a $op y.a)
    # because we cannot interpolate expressions in fixed notations/syntax.
end

In [33]:
MyInteger(1) + MyInteger(2) # MyInteger(3)

MyInteger(3)

## 2. Expressions

Expression (`Expr`) is a data type (structure) in Julia to represent (a piece of) Julia code. It must be given in valid Julia syntax.

### 2.1 Expression structure

In [34]:
ex = :(a + 1)
# pieces in the expression "a + 1" can be modified

:(a + 1)

In [36]:
# dump() to see the structure of an expression (syntax tree)
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol a
    3: Int64 1


In [51]:
ex = :((a + b)*c)
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol *
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Symbol a
        3: Symbol b
    3: Symbol c


In [45]:
ex.head # :call
ex.args # 3-element Array{Any,1}: :(a + b) : * :c

3-element Vector{Any}:
 :*
 :(a + b)
 :c

In [52]:
# change the pieces of the expression
ex.args[2].args[1]  = :- # change + to -
ex

:((a - b) * c)

In [1]:
# Invalid syntax causes error in the evaluation
ex = :( 1 + 1 )
# dump(ex)
ex.args[1] = 1
eval(ex) # MethodError: objects of type Int64 are not callable

MethodError: MethodError: objects of type Int64 are not callable
Maybe you forgot to use an operator such as *, ^, %, / etc. ?

### 2.2 Evaluate an expression

In [20]:
ex = :( isapprox(exp(im * pi) + 1, 0.0, atol=1e-10) )
eval(ex) # true, eval interprets the value of ex as a Julia expression 
@eval $ex # true
@eval isapprox(exp(im * pi) + 1, 0.0, atol=1e-10) # true
@eval ex # :( isapprox(exp(im * pi) + 1, 0.0, atol = 1.0e-10) )
# @eval executes "ex" as a code having been interpolated
# @eval can be used in local scope, i.e. within a function

:(isapprox(exp(im * pi) + 1, 0.0, atol = 1.0e-10))

In [1]:
ex =
quote # for long peices of expressions
    z = x + y
end

quote
    #= c:\Projects\julia-introduction\notebooks\Bonus_metaprogramming.ipynb:3 =#
    z = x + y
end

In [2]:
typeof(ex)

Expr

In [16]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((4,))
    1: Symbol isapprox
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Expr
          head: Symbol call
          args: Array{Any}((2,))
            1: Symbol exp
            2: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol *
                2: Symbol im
                3: Symbol pi
        3: Int64 1
    3: Float64 0.0
    4: Expr
      head: Symbol kw
      args: Array{Any}((2,))
        1: Symbol atol
        2: Float64 1.0e-10


In [3]:
ex.head

:block

In [4]:
ex.args

2-element Vector{Any}:
 :(#= c:\Projects\julia-introduction\notebooks\Bonus_metaprogramming.ipynb:3 =#)
 :(z = x + y)

Can also be written as:

In [63]:
ex = :(z = x + y)

:(z = x + y)

In [6]:
x = 1
y = 2
eval(ex)

3

In [7]:
ex

:(z = x + y)

## Manipulating expressions

In [8]:
ex = :(3x + 2x^2)

:(3x + 2 * x ^ 2)

Let's replace all `x` by `(x+1)`s.

In [9]:
ex.args

3-element Vector{Any}:
 :+
 :(3x)
 :(2 * x ^ 2)

In [16]:
ex.args[2].args[3]

:x

In [17]:
ex.args[2].args[3] = :(x+1)

:(x + 1)

In [52]:
ex

:(3x + 3 * x ^ 2)

In [51]:
eval(ex)

6

## Parsing from text

In [43]:
s = "3x + 3x^2"

"3x + 3x^2"

In [44]:
ex = Meta.parse(s)

:(3x + 3 * x ^ 2)

In [47]:
x=1
eval(ex)

6

## Macros

@ is the signal that it's a "macro", i.e. a meta-function.

A function that takes as its argument a piece of code, manipulates it, and returns the resulting piece of code (returns an expression that is also executed).

What a macro does
- 1. stores the input code as an expression
- 2. manipulates the expression: add code before, after, or replace pieces of the input code
- 3. returns the resulting expression
- 4. evaluate the expression, i.e. run the code

## Wilkinson polynomial
$$p_n(x) = \prod_{i=1}^n (x - i)$$
E.g.
$$p_5(x) = (x-1)(x-2)(x-3)(x-4)(x-5)$$

In [21]:
function wilkinson(n, a)
    return prod([x - a for x in 1:n])
end

wilkinson (generic function with 1 method)

In [34]:
macro make_wilkinson(n, name)
    ex = :( x - 1 )
    for i in 2:n
        ex = :( $ex * (x - $i) )
        # Everything within :() is treated as a symbol until being evaluated, 
        # unless the element is interpolated with $ beforehand.
        # $ is used to interpolate the value of a variable (the content represented by "ex" and "i") inside the expression :().
        # Otherwise, the variable "ex" and "i" within the expression is treated as a symbol.
    end
    # to obtain the expression p_n(x) = (x - 1) * (x - 2) * ... * (x - n) with name
    :( $(name)(x) = $ex )
end

@make_wilkinson (macro with 1 method)

In [38]:
@make_wilkinson 6 p6
p6(0.1)

559.122291